In [1]:
DEVICE = 'cuda:2'
BATCH_SIZE = 256

In [2]:
import os
import sys
import torch
import torchvision.datasets as Datasets
from torchvision import transforms as tfms
from torch.utils.data import DataLoader
sys.path.append("../../")
torch.cuda.set_device(int(DEVICE[-1]))

In [3]:
stats = ((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
train_tfms = tfms.Compose([
    tfms.RandomCrop(32, padding=4, padding_mode='reflect'),
    tfms.RandomHorizontalFlip(),
    tfms.ToTensor(),
    tfms.Normalize(*stats, inplace=True)
])
test_tfms = tfms.Compose([
    tfms.ToTensor(),
    tfms.Normalize(*stats)
])

In [4]:
cifar100_train = Datasets.CIFAR100(root='./data', train=True, download=True, transform=test_tfms)
cifar100_test = Datasets.CIFAR100(root='./data', train=False, download=True, transform=test_tfms)

train_loader = DataLoader(cifar100_train, shuffle=True, num_workers=1, batch_size=BATCH_SIZE)
test_loader = DataLoader(cifar100_test, shuffle=False, num_workers=1, batch_size=2*BATCH_SIZE)

dataloaders = {"train" : train_loader , "val" : test_loader}

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# import libraries
from trailmet.models import resnet
from trailmet.algorithms.quantize.lapq import LAPQ

In [6]:
# load model
cnn=resnet.make_resnet50(100,32)
checkpoint = torch.load("./resnet50_cifar100-pretrained.pth", map_location=DEVICE)
cnn.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [7]:
# test model
from trailmet.algorithms.algorithms import BaseAlgorithm
BaseAlgorithm().test(model=cnn, dataloader=test_loader, device=torch.device(DEVICE))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.95it/s, acc1=72.5, acc5=91.5]


(72.53963699340821, 91.5182674407959)

In [ ]:
# quantize model
kwargs = {
    'W_BITS':4, 
    'A_BITS':8, 
    'ACT_QUANT':True,
    'CALIB_BATCHES':1024//BATCH_SIZE, 
    'MAX_ITER':1,
    'MAX_FEV':1,
    'VERBOSE':True,
    'PRINT_FREQ':50,
    'GPU_ID':int(DEVICE[-1]),
    'SEED':42
    }
qnn = LAPQ(cnn, dataloaders, **kwargs)
qnn.compress_model()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.83it/s, acc1=12.6, acc5=31.9]


Quantization (W4A8) accuracy before LAPQ: (12.637867641448974, 31.852021980285645)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.31it/s, acc1=66.7, acc5=87.3, loss=1.45]


==> p intr : 3.16
==> loss : 1.4545
==> acc@1 | acc@5 : 66.6843 | 87.2691
==> iteration: 0, minimum loss so far: 0.2761
==> iteration: 50, minimum loss so far: 0.2543
==> iteration: 100, minimum loss so far: 0.2207
==> iteration: 150, minimum loss so far: 0.2196
==> iteration: 200, minimum loss so far: 0.2193
==> iteration: 250, minimum loss so far: 0.2178
==> iteration: 300, minimum loss so far: 0.2154
==> iteration: 350, minimum loss so far: 0.2117
==> iteration: 400, minimum loss so far: 0.2111
==> iteration: 450, minimum loss so far: 0.2045
==> iteration: 500, minimum loss so far: 0.2045
==> iteration: 550, minimum loss so far: 0.2044
==> iteration: 600, minimum loss so far: 0.2042
==> iteration: 650, minimum loss so far: 0.2030
==> iteration: 700, minimum loss so far: 0.2004
==> iteration: 750, minimum loss so far: 0.1984
==> iteration: 800, minimum loss so far: 0.1943
==> iteration: 850, minimum loss so far: 0.1941
==> iteration: 900, minimum loss so far: 0.1941
==> iteration: 95